In [1]:
%matplotlib inline

import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

import json
import glob
import os

In [4]:
paths = glob.glob("/home/zeitz/experiments/chpot_journal/bidir/*.json")
data = [json.load(open(path)) for path in paths]

In [5]:
queries = pd.DataFrame.from_records([{
    **algo,
    'experiment': exp['experiment'],
    'factor': exp['factor'],
    'probability': exp.get('probability', 1),
    'speed': exp.get('speed', 0),
    'potential': run['potential'],
    'num_queue_pops': algo['num_queue_pops'] + algo.get('core_search', {}).get('num_queue_pops', 0),
    'num_queue_pushs': algo['num_queue_pushs'] + algo.get('core_search', {}).get('num_queue_pushs', 0),
    'num_relaxed_arcs': algo['num_relaxed_arcs'] + algo.get('core_search', {}).get('num_relaxed_arcs', 0),
    }
    for run in data for exp in run['experiments'] for algo in exp['algo_runs']])

queries['choose_direction_strategy'] = queries['choose_direction_strategy'].fillna('unidir')
queries['bidir_pot'] = queries['bidir_pot'].fillna('unidir')
queries['improved_pruning'] = queries['improved_pruning'].fillna('unidir')

In [10]:
pd.set_option('display.max_rows', 500)

In [61]:
queries.groupby(['potential', 'experiment', 'factor', 'probability', 'algo', 'bidir_pot', 'choose_direction_strategy', 'improved_pruning'])[['running_time_ms', 'num_queue_pops']].median().unstack(0)

running_time_ms  \
potential                                                                                                                                                   CH   
experiment                   factor probability algo                                      bidir_pot choose_direction_strategy improved_pruning                   
probabilistic_scale_by_speed 1.25   0.00        Bidrectional Dijkstra Query               Average   alternating               False                     417.10   
                                                                                                                              True                      428.60   
                                                                                                    min_key                   False                     443.94   
                                                                                                                              True                      450.67   
                                                                                          Symmetric alternating               False                     798.01   
                                                                                                                              True                      406.77   
                                                                                                    min_key                   False                     797.63   
                                                                                                                              True                      441.74   
                                                Dijkstra Query                            unidir    unidir                    unidir                    256.06   
                                                Virtual Topocore Bidirectional Core Query Average   alternating               False                     120.05   
                                                                                                                              True                      122.51   
                                                                                                    min_key                   False                     137.20   
                                                                                                                              True                      140.46   
                                                                                          Symmetric alternating               False                     252.41   
                                                                                                                              True                      134.88   
                                                                                                    min_key                   False                     268.94   
                                                                                                                              True                      154.96   
                                                Virtual Topocore Component Query          unidir    unidir                    unidir                     93.92   
                                    1.00        Bidrectional Dijkstra Query               Average   alternating               False                       6.42   
                                                                                                                              True                        6.57   
                                                                                                    min_key                   False                      12.17   
                                                                                                                              True                       12.40   
                                                                                          Sym

In [62]:
queries.loc[lambda x: x['choose_direction_strategy'] != 'min_key'] \
    .loc[lambda x: ~(((x['bidir_pot'] == 'Average') & (x['improved_pruning'] == True)) | ((x['bidir_pot'] == 'Symmetric') & (x['improved_pruning'] == False)))] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'algo', 'bidir_pot', 'improved_pruning'])[['running_time_ms', 'num_queue_pops']].median().unstack(0) \
    .loc[('weight_scale', 1.05, 1.00)] \
    .droplevel(2)

running_time_ms  \
potential                                                        CH   
algo                                      bidir_pot                   
Bidrectional Dijkstra Query               Average             36.78   
                                          Symmetric           35.74   
Dijkstra Query                            unidir              26.51   
Virtual Topocore Bidirectional Core Query Average             12.18   
                                          Symmetric           13.44   
Virtual Topocore Component Query          unidir               9.09   

                                                    num_queue_pops  
potential                                                       CH  
algo                                      bidir_pot                 
Bidrectional Dijkstra Query               Average        75,825.00  
                                          Symmetric      76,894.00  
Dijkstra Query                            unidir         75,810.50  
Virtual Topocore Bidirectional Core Query Average        12,079.00  
                                          Symmetric      13,051.00  
Virtual Topocore Component Query          unidir         13,097.50

### Effectiveness new pruning

In [63]:
queries.loc[lambda x: x['choose_direction_strategy'] != 'min_key'] \
    .loc[lambda x: x['bidir_pot'] != 'unidir'] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'algo', 'bidir_pot', 'improved_pruning'])[['running_time_ms', 'num_queue_pops']].median().unstack(0) \
    .loc[('weight_scale', 1.05, 1.00)]

running_time_ms  \
potential                                                                         CH   
algo                                      bidir_pot improved_pruning                   
Bidrectional Dijkstra Query               Average   False                      36.78   
                                                    True                       37.43   
                                          Symmetric False                      66.35   
                                                    True                       35.74   
Virtual Topocore Bidirectional Core Query Average   False                      12.18   
                                                    True                       12.30   
                                          Symmetric False                      23.10   
                                                    True                       13.44   

                                                                     num_queue_pops  
potential                                                                        CH  
algo                                      bidir_pot improved_pruning                 
Bidrectional Dijkstra Query               Average   False                 75,825.00  
                                                    True                  75,825.00  
                                          Symmetric False                157,427.00  
                                                    True                  76,894.00  
Virtual Topocore Bidirectional Core Query Average   False                 12,079.00  
                                                    True                  12,075.00  
                                          Symmetric False                 25,599.00  
                                                    True                  13,051.00

### Min key is Quatsch

In [67]:
queries.loc[lambda x: x['bidir_pot'] != 'unidir'] \
    .loc[lambda x: x['algo'].str.contains('Topocore')] \
    .loc[lambda x: ~(((x['bidir_pot'] == 'Average') & (x['improved_pruning'] == True)) | ((x['bidir_pot'] == 'Symmetric') & (x['improved_pruning'] == False)))] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'bidir_pot', 'choose_direction_strategy'])[['running_time_ms', 'num_queue_pops']].median().unstack(0) \
    .loc[('probabilistic_scale_by_speed', 1.50)]

running_time_ms num_queue_pops
potential                                                    CH             CH
probability bidir_pot choose_direction_strategy                               
0.00        Average   alternating                        270.67     289,559.00
                      min_key                            290.48     313,774.50
            Symmetric alternating                        295.96     299,884.00
                      min_key                            311.47     324,250.00
1.00        Average   alternating                          4.84       5,125.00
                      min_key                              8.36       8,500.00
            Symmetric alternating                          5.41       5,865.00
                      min_key                             10.17      10,088.50

### Bidir Effectiveness

In [69]:
queries.loc[lambda x: x['algo'].str.contains('Topocore')] \
    .loc[lambda x: x['choose_direction_strategy'] != 'min_key'] \
    .loc[lambda x: ~(((x['bidir_pot'] == 'Average') & (x['improved_pruning'] == True)) | ((x['bidir_pot'] == 'Symmetric') & (x['improved_pruning'] == False)))] \
    .groupby(['potential', 'experiment', 'factor', 'probability', 'bidir_pot'])[['running_time_ms', 'num_queue_pops']].median().unstack(0)

running_time_ms  \
potential                                                              CH   
experiment                   factor probability bidir_pot                   
probabilistic_scale_by_speed 1.25   0.00        Average            120.05   
                                                Symmetric          134.88   
                                                unidir              93.92   
                                    1.00        Average              2.40   
                                                Symmetric            2.66   
                                                unidir               3.27   
                             1.50   0.00        Average            270.67   
                                                Symmetric          295.96   
                                                unidir             214.35   
                                    1.00        Average              4.84   
                                                Symmetric            5.41   
                                                unidir               8.90   
weight_scale                 1.00   1.00        Average              0.82   
                                                Symmetric            0.65   
                                                unidir               0.45   
                             1.05   1.00        Average             12.18   
                                                Symmetric           13.44   
                                                unidir               9.09   
                             1.10   1.00        Average             38.48   
                                                Symmetric           43.62   
                                                unidir              28.47   

                                                          num_queue_pops  
potential                                                             CH  
experiment                   factor probability bidir_pot                 
probabilistic_scale_by_speed 1.25   0.00        Average       127,872.00  
                                                Symmetric     132,596.00  
                                                unidir        122,716.00  
                                    1.00        Average         2,375.00  
                                                Symmetric       2,671.00  
                                                unidir          5,668.50  
                             1.50   0.00        Average       289,559.00  
                                                Symmetric     299,884.00  
                                                unidir        297,381.50  
                                    1.00        Average         5,125.00  
                                                Symmetric       5,865.00  
                                                unidir         14,604.00  
weight_scale                 1.00   1.00        Average           231.00  
                                                Symmetric         234.00  
                                                unidir            232.00  
                             1.05   1.00        Average        12,079.00  
                                                Symmetric      13,051.00  
                                                unidir         13,097.50  
                             1.10   1.00        Average        39,330.00  
                                                Symmetric      40,761.00  
                                                unidir         38,140.00